In [1]:
import sys
import os

# 设置你的 main.py 所在目录路径，例如：
project_dir = "/common/users/sl2148/Public/yang_ouyang/alignment-attribution-code/"

# 自动获取当前 Notebook 所在目录下的 "src" 或其他文件夹
# project_dir = os.path.abspath("")

# 加入到 sys.path（如果尚未添加）
if project_dir not in sys.path:
    sys.path.append(project_dir)

# 检查是否添加成功
print("Updated sys.path:", sys.path)

Updated sys.path: ['/common/users/sl2148/Public/yang_ouyang/alignment-attribution-code/scripts', '/common/home/sl2148/anaconda3/envs/prune_llm/lib/python39.zip', '/common/home/sl2148/anaconda3/envs/prune_llm/lib/python3.9', '/common/home/sl2148/anaconda3/envs/prune_llm/lib/python3.9/lib-dynload', '', '/common/home/sl2148/anaconda3/envs/prune_llm/lib/python3.9/site-packages', '/common/users/sl2148/Public/yang_ouyang/alignment-attribution-code/']


In [2]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from lib.prune import (
    prune_wanda,
    check_sparsity,
    get_mask,
)
from lib.eval import eval_ppl, eval_zero_shot, eval_attack
from vllm import LLM
import argparse

In [3]:
# 📌 设置参数（你原本命令行中给出的内容）
# 构造参数
args = argparse.Namespace(
    model="llama2-7b-chat-hf",
    model_base="llama2-7b-hf",
    seed=0,
    nsamples=128,
    sparsity_ratio=0.5,
    sparsity_type="unstructured",
    prune_method="wanda",  # 举例使用 wanda
    prune_data="alpaca_cleaned_no_safety",
    use_diff=False,
    neg_prune=False,
    recover_from_base=False,
    p=0.5,
    q=0.5,
    top_k_heads=10,
    cache_dir="llm_weights",
    use_variant=False,
    save=None,
    save_model=None,
    save_mask=None,
    dump_wanda_score=False,
    eval_zero_shot=False,
    eval_attack=False,
    save_attack_res=False,
    prune_part=False,
    disentangle=True,  # 注意：原 argparse 中是 --entangle_prompt_feat -> dest="disentangle", action="store_false"
    decouple_align_utility=False,
    decouple_align_misalign=False,
    rank=10,
    niter=20,
)
model_name = "llama2-7b-chat-hf"
model_base = "llama2-7b-hf"
prune_method = "wanda"
sparsity_ratio = 0.5
sparsity_type = "unstructured"
prune_data = "align"
neg_prune = True
save_attack_res = True
eval_zero_shot = True
eval_attack = True
suffix = "weightonly"
cache_dir = "llm_weights"
save_dir = f"out/{model_name}/{sparsity_type}/{prune_method}_{suffix}/{prune_data}"
os.makedirs(save_dir, exist_ok=True)

In [4]:
# 💾 模型路径映射
modeltype2path = {
    "llama2-7b-chat-hf": "meta-llama/Llama-2-7b-chat-hf",
    "llama2-7b-hf": "meta-llama/Llama-2-7b-hf",
}

# ✅ 加载模型和 tokenizer
def get_llm(model_name, cache_dir):
    model = AutoModelForCausalLM.from_pretrained(
        modeltype2path[model_name],
        torch_dtype=torch.bfloat16,
        cache_dir=cache_dir,
        low_cpu_mem_usage=True,
        device_map="auto",
        token=os.environ.get("HF_TOKEN"),
    )
    model.seqlen = model.config.max_position_embeddings
    return model

model = get_llm(model_name, cache_dir)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [ ]:
# tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    modeltype2path[model_name],
    use_fast=False,
    cache_dir=cache_dir,
)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Pruning (仅 unstructured 的 wanda 方法)
prune_n, prune_m = 0, 0
print("开始剪枝：", prune_method)
prune_wanda(
    args=args,  # 不用 args
    model=model,
    tokenizer=tokenizer,
    model_base=model_base,  
    device=device,
    prune_n=prune_n,
    prune_m=prune_m,
    prune_data=args.prune_data,
)

# ✅ Sparsity 检查
sparsity_ratio_actual = check_sparsity(model)
print(f"实际 sparsity: {sparsity_ratio_actual:.6f}")

# ✅ PPL 评估
ppl = eval_ppl(
    args=None,
    model=model,
    tokenizer=tokenizer,
    device=device,
)
print(f"Perplexity (Wikitext): {ppl:.4f}")

# ✅ 选择性保存 mask（可选）
save_mask_flag = False
if save_mask_flag:
    mask = get_mask(model, neg_prune)
    mask_dir = os.path.join(save_dir, "FT_mask")
    os.makedirs(mask_dir, exist_ok=True)
    mask_path = os.path.join(mask_dir, f"mask_bottom_{sparsity_ratio:.3f}.pt")
    torch.save(mask, mask_path)
    print(f"Mask 已保存: {mask_path}")


Using device: cuda:0
开始剪枝： wanda
loading calibration data alpaca_cleaned_no_safety
dataset loading complete
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([29, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([144, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([349, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([42, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([691, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([489, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([85, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([240, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([67, 4096])
[DEBUG] inp type: <class 'torch.Tensor'>, len: 1, first shape: torch.Size([90, 4096])
[DEBUG] inp type: <class 't

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

nsamples 83
sample 0
sample 50
Perplexity (Wikitext): 8.0701
